In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_json
from pyspark.sql.types import StructType, StringType, DoubleType, TimestampType

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("CryptoDataStreaming") \
    .master("local[*]") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1") \
    .getOrCreate()



24/03/29 14:07:32 WARN Utils: Your hostname, Priyas-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.68.59 instead (on interface en0)
24/03/29 14:07:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/priyasuresh/.ivy2/cache
The jars for the packages stored in: /Users/priyasuresh/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f6a68627-89f3-4544-ab8a-475611475645;1.0
	confs: [default]


:: loading settings :: url = jar:file:/Users/priyasuresh/.pyenv/versions/3.10.4/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.1 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
:: resolution report :: resolve 205ms :: artifacts dl 7ms
	:: modules in use:
	com.google.code.findbugs#jsr305;3.0.0 from central in [default]
	commons-logging#commons-logging;1.1.3 from central in [default]
	org.apache.commons#commons-pool2;2.11.1 from central in [default]
	org.apache.hadoop#hadoop-client-api;3.3.4 from central in [default]
	org.apache.hadoop#h

In [2]:
# Define the schema of the message
# Adjust this schema according to the actual structure of your messages
messageSchema = StructType() \
    .add("symbol_id", StringType()) \
    .add("sequence", StringType()) \
    .add("time_exchange", TimestampType()) \
    .add("time_coinapi", TimestampType()) \
    .add("uuid", StringType()) \
    .add("price", DoubleType()) \
    .add("size", DoubleType()) \
    .add("taker_side", StringType())

# Subscribe to multiple topics
topic_names = "bitstamp_btc_usd,bitfinex_btc_usd,kraken_btc_usd"

# Create a streaming DataFrame that reads from the topics
df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", topic_names) \
    .option("startingOffsets", "earliest") \
    .load()

# Deserialize the JSON data and apply the schema
value_df = df.select(from_json(col("value").cast("string"), messageSchema).alias("data")).select("data.*")

# Print the streaming DataFrame to the console
query = value_df \
    .writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

# Wait for the stream to finish
query.awaitTermination()


24/03/29 14:07:40 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/9j/0m5g2l6d03z4pd5y6pt7xh1h0000gn/T/temporary-3cf4de83-faba-482e-907a-24e21d6202ce. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/03/29 14:07:40 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/03/29 14:07:41 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


-------------------------------------------
Batch: 0
-------------------------------------------
+--------------------+--------+--------------------+--------------------+--------------------+-------+----------+----------+
|           symbol_id|sequence|       time_exchange|        time_coinapi|                uuid|  price|      size|taker_side|
+--------------------+--------+--------------------+--------------------+--------------------+-------+----------+----------+
|BITFINEX_SPOT_BTC...|   10003|2024-03-29 13:11:...|2024-03-29 13:11:...|461da737-1a58-471...|69644.0|      0.02|      SELL|
|BITFINEX_SPOT_BTC...|   10004|2024-03-29 13:11:...|2024-03-29 13:11:...|461da737-1a58-471...|69644.0|      0.02|      SELL|
|BITFINEX_SPOT_BTC...|   10005|2024-03-29 13:11:...|2024-03-29 13:11:...|e0e299ca-8839-4ed...|69615.0|      0.01|       BUY|
|BITFINEX_SPOT_BTC...|   10006|2024-03-29 13:11:...|2024-03-29 13:11:...|e0e299ca-8839-4ed...|69615.0|      0.01|       BUY|
|BITFINEX_SPOT_BTC...|   416

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/Users/priyasuresh/.pyenv/versions/3.10.4/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/priyasuresh/.pyenv/versions/3.10.4/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/Users/priyasuresh/.pyenv/versions/3.10.4/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

-------------------------------------------
Batch: 8
-------------------------------------------
+--------------------+--------+--------------------+--------------------+--------------------+-------+-----+----------+
|           symbol_id|sequence|       time_exchange|        time_coinapi|                uuid|  price| size|taker_side|
+--------------------+--------+--------------------+--------------------+--------------------+-------+-----+----------+
|BITFINEX_SPOT_BTC...|   43168|2024-03-29 14:07:...|2024-03-29 14:07:...|c7030a3e-513c-40e...|69538.0|0.005|      SELL|
|BITFINEX_SPOT_BTC...|   43169|2024-03-29 14:07:...|2024-03-29 14:07:...|c7030a3e-513c-40e...|69538.0|0.005|      SELL|
+--------------------+--------+--------------------+--------------------+--------------------+-------+-----+----------+

-------------------------------------------
Batch: 9
-------------------------------------------
+--------------------+--------+--------------------+--------------------+------------

In [3]:
query.stop()

-------------------------------------------
Batch: 10
-------------------------------------------
+-------------------+--------+--------------------+--------------------+--------------------+-------+---------+----------+
|          symbol_id|sequence|       time_exchange|        time_coinapi|                uuid|  price|     size|taker_side|
+-------------------+--------+--------------------+--------------------+--------------------+-------+---------+----------+
|KRAKEN_SPOT_BTC_USD|   24875|2024-03-29 14:08:...|2024-03-29 14:08:...|e2773d25-d093-47a...|69553.5|2.6821E-4|      SELL|
|KRAKEN_SPOT_BTC_USD|   24876|2024-03-29 14:08:...|2024-03-29 14:08:...|e2773d25-d093-47a...|69553.5|2.6821E-4|      SELL|
+-------------------+--------+--------------------+--------------------+--------------------+-------+---------+----------+



In [4]:
spark.stop()

In [5]:
import numpy as np
import hashlib
from pyspark.sql.functions import col, from_json
from pyspark.sql import SparkSession
from pyspark.sql.types import *

# Define SimpleFilter class
class SimpleFilter:
    def __init__(self, size, hash_count):
        self.size = size
        self.hash_count = hash_count
        self.bit_array = np.zeros(size, dtype=np.bool_)

    def _hash(self, item, seed):
        hash_val = int(hashlib.md5(item.encode('utf-8')).hexdigest(), 16)
        return (hash_val + seed) % self.size

    def add(self, item):
        for seed in range(self.hash_count):
            index = self._hash(item, seed)
            self.bit_array[index] = True

    def check(self, item):
        for seed in range(self.hash_count):
            index = self._hash(item, seed)
            if not self.bit_array[index]:
                return False
        return True

# Initialize Spark Session
#spark = SparkSession.builder.appName("StreamingWithBloomFilter").getOrCreate()
spark = SparkSession.builder \
    .appName("StreamingWithBloomFilter") \
    .master("local[*]") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1") \
    .getOrCreate()


# Define the schema of the message
messageSchema = StructType([
    StructField("symbol_id", StringType()),
    StructField("sequence", StringType()),
    StructField("time_exchange", TimestampType()),
    StructField("time_coinapi", TimestampType()),
    StructField("uuid", StringType()),
    StructField("price", DoubleType()),
    StructField("size", DoubleType()),
    StructField("taker_side", StringType())
])

# Subscribe to multiple topics
topic_names = "bitstamp_btc_usd,bitfinex_btc_usd,kraken_btc_usd"

# Create a streaming DataFrame that reads from the topics
df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", topic_names) \
    .option("startingOffsets", "earliest") \
    .load()

# Deserialize the JSON data and apply the schema
value_df = df.select(from_json(col("value").cast("string"), messageSchema).alias("data")).select("data.*")

# Initialize the Bloom Filter
# Assuming a capacity of 10000 and 3 hash functions for simplicity
bloom_filter = SimpleFilter(10000, 3)

def process_batch(batch_df, batch_id):
    # Convert to Pandas DataFrame for easier manipulation
    pandas_df = batch_df.toPandas()
    
    # Apply the Bloom Filter to each row
    for index, row in pandas_df.iterrows():
        uuid = row['uuid']
        if not bloom_filter.check(uuid):
            bloom_filter.add(uuid)  # Use add method to insert UUID into the Bloom filter
            # Process the row since it's not a duplicate
            print(row)

# Apply the function to each micro-batch
query = value_df.writeStream.foreachBatch(process_batch).start()

# Wait for the stream to finish
query.awaitTermination()


24/03/29 14:08:47 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/9j/0m5g2l6d03z4pd5y6pt7xh1h0000gn/T/temporary-c9f7063a-db5a-4d51-be3c-efd6e3d266f8. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/03/29 14:08:47 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/03/29 14:08:47 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


symbol_id                      BITFINEX_SPOT_BTC_USDT
sequence                                        10003
time_exchange              2024-03-29 13:11:39.714000
time_coinapi               2024-03-29 13:11:39.733197
uuid             461da737-1a58-471c-97bf-0a149b60acb0
price                                         69644.0
size                                             0.02
taker_side                                       SELL
Name: 0, dtype: object
symbol_id                      BITFINEX_SPOT_BTC_USDT
sequence                                        10005
time_exchange              2024-03-29 13:11:44.332000
time_coinapi               2024-03-29 13:11:44.358019
uuid             e0e299ca-8839-4ed5-8f65-02bc55acafa9
price                                         69615.0
size                                             0.01
taker_side                                        BUY
Name: 2, dtype: object
symbol_id                       BITFINEX_SPOT_BTC_USD
sequence                            

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/Users/priyasuresh/.pyenv/versions/3.10.4/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
RuntimeError: reentrant call inside <_io.BufferedReader name=80>

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/priyasuresh/.pyenv/versions/3.10.4/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/priyasuresh/.pyenv/versions/3.10.4/lib/python3.10/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/Users/priyasuresh/.pyenv/versions/3.10.4/lib/python3.10/site-packages/py4j/clients

Py4JError: An error occurred while calling o105.awaitTermination

symbol_id                      BITFINEX_SPOT_BTC_USDT
sequence                                        10646
time_exchange              2024-03-29 14:09:00.294000
time_coinapi               2024-03-29 14:09:00.310141
uuid             92bc9633-1d12-45af-80e6-da5f5b551bf8
price                                         69500.0
size                                         0.000645
taker_side                                       SELL
Name: 0, dtype: object


symbol_id                         KRAKEN_SPOT_BTC_USD
sequence                                        24903
time_exchange              2024-03-29 14:09:01.225194
time_coinapi               2024-03-29 14:09:01.246297
uuid             92e01e5f-e346-426b-8138-1d4f29c7d528
price                                         69549.5
size                                         0.014231
taker_side                                       SELL
Name: 0, dtype: object
symbol_id                      BITFINEX_SPOT_BTC_USDT
sequence                                        10648
time_exchange              2024-03-29 14:09:01.794000
time_coinapi               2024-03-29 14:09:01.815680
uuid             d2e32ddc-f4f7-4b86-8361-459889aa7979
price                                         69500.0
size                                         0.000325
taker_side                                       SELL
Name: 0, dtype: object
symbol_id                      BITFINEX_SPOT_BTC_USDT
sequence                            

symbol_id                      BITFINEX_SPOT_BTC_USDT
sequence                                        10652
time_exchange              2024-03-29 14:09:02.843000
time_coinapi               2024-03-29 14:09:02.865626
uuid             b843490f-6d03-404d-8373-137c9c46b18e
price                                         69500.0
size                                         0.000871
taker_side                                       SELL
Name: 0, dtype: object
symbol_id                        KRAKEN_SPOT_BTC_USDT
sequence                                         5533
time_exchange              2024-03-29 14:09:07.255954
time_coinapi               2024-03-29 14:09:07.290916
uuid             2fbe3d4b-82fd-4c48-b5b0-8c002388582a
price                                         69496.4
size                                             0.05
taker_side                                        BUY
Name: 0, dtype: object
symbol_id                       BITSTAMP_SPOT_BTC_USD
sequence                            

symbol_id                        KRAKEN_SPOT_BTC_USDT
sequence                                         5537
time_exchange              2024-03-29 14:09:17.239490
time_coinapi               2024-03-29 14:09:17.284863
uuid             9f88d85f-25db-4fb7-9994-818fe50dd242
price                                         69501.7
size                                         0.002876
taker_side                                        BUY
Name: 0, dtype: object
symbol_id                         KRAKEN_SPOT_BTC_USD
sequence                                        24907
time_exchange              2024-03-29 14:09:18.346935
time_coinapi               2024-03-29 14:09:18.368911
uuid             8b11c1e6-1671-456f-b66d-69068540e522
price                                         69549.6
size                                         0.004114
taker_side                                        BUY
Name: 0, dtype: object
symbol_id                       BITSTAMP_SPOT_BTC_USD
sequence                            

In [6]:
query.stop()

In [7]:
import numpy as np
import hashlib
from pyspark.sql.functions import col, from_json
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from scipy.stats import laplace  # Import Laplace distribution for differential privacy

# Define SimpleFilter class
class SimpleFilter:
    def __init__(self, size, hash_count):
        self.size = size
        self.hash_count = hash_count
        self.bit_array = np.zeros(size, dtype=np.bool_)

    def _hash(self, item, seed):
        hash_val = int(hashlib.md5(item.encode('utf-8')).hexdigest(), 16)
        return (hash_val + seed) % self.size

    def add(self, item):
        for seed in range(self.hash_count):
            index = self._hash(item, seed)
            self.bit_array[index] = True

    def check(self, item):
        for seed in range(self.hash_count):
            index = self._hash(item, seed)
            if not self.bit_array[index]:
                return False
        return True

from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, DoubleType
from scipy.stats import laplace

# Initialize Spark session
spark = SparkSession \
    .builder \
    .appName("StreamingWithBloomFilter") \
    .master("local[*]") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1") \
    .getOrCreate()

# Define the schema of the message
messageSchema = StructType([
    StructField("symbol_id", StringType()),
    StructField("sequence", StringType()),
    StructField("time_exchange", TimestampType()),
    StructField("time_coinapi", TimestampType()),
    StructField("uuid", StringType()),
    StructField("price", DoubleType()),
    StructField("size", DoubleType()),
    StructField("taker_side", StringType())
])

# Subscribe to multiple topics
topic_names = "bitstamp_btc_usd,bitfinex_btc_usd,kraken_btc_usd"

# Create a streaming DataFrame that reads from the topics
df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", topic_names) \
    .load()

# Deserialize the JSON data and apply the schema
value_df = df.select(from_json(col("value").cast("string"), messageSchema).alias("data")).select("data.*")

# Initialize the Bloom Filter
bloom_filter = SimpleFilter(10000, 3)

# Function to add Laplacian noise for differential privacy
def add_laplace_noise(data, epsilon=1.0, sensitivity=1):
    try:
        # Convert data to float and then add Laplace noise
        data_float = float(data)
        scale = sensitivity / epsilon
        noise = laplace.rvs(scale=scale, size=1)[0]
        return str(data_float + noise)  # Convert back to string after adding noise
    except ValueError:
        # If data cannot be converted to float (e.g., UUID), return the original value
        return data

# Define process_batch function with differential privacy
def process_batch(batch_df, batch_id):
    # Convert to Pandas DataFrame for easier manipulation
    pandas_df = batch_df.toPandas()
    
    # Apply differential privacy to each UUID
    pandas_df['uuid'] = pandas_df['uuid'].apply(add_laplace_noise)
    
    # Apply the Bloom Filter to each row
    for index, row in pandas_df.iterrows():
        uuid = row['uuid']
        if not bloom_filter.check(uuid):
            bloom_filter.add(uuid)  # Use add method to insert UUID into the Bloom filter
            # Process the row since it's not a duplicate
            print(row)

# Apply the function to each micro-batch
query = value_df.writeStream.foreachBatch(process_batch).start()

# Wait for the stream to finish
query.awaitTermination()



24/03/29 14:09:55 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/9j/0m5g2l6d03z4pd5y6pt7xh1h0000gn/T/temporary-9b6f46a1-be76-40d3-a9a0-fc2b9e21023d. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/03/29 14:09:55 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/03/29 14:09:55 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.
ERROR:root:KeyboardInterrupt while sending command.                             
Traceback (most recent call last):
  File "/Users/priyasuresh/.pyenv/versions/3.10.4/lib/python3.10/site-packages/py4j/java_gateway.py", line

KeyboardInterrupt: 